# Project

 ## Product Rating

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import torch
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")


# Carica il primo dataset
dataset1 = pd.read_csv("Tiny Eco Wonderflow3.csv", sep=";", on_bad_lines="skip")  # Prova con ";" come separatore
dataset1.columns = dataset1.columns.str.strip()

# Estrai la colonna 'text' dal primo dataset
texts1 = dataset1['text'].dropna()
texts1 = texts1[texts1.str.strip() != '']

# Rimuovi gli spazi all'inizio e alla fine del testo, e le nuove righe extra nel primo dataset
texts1_cleaned = texts1.str.replace(r'\s+', ' ', regex=True)  # Rimuove spazi extra (compresi \n e \t)
texts1 = texts1_cleaned.str.strip()  # Rimuove gli spazi ai bordi

###

# Carica il secondo dataset
dataset2 = pd.read_csv("Tiny Eco Digimind.csv", sep=";", on_bad_lines="skip") 
dataset2.columns = dataset2.columns.str.strip()

# Estrai e concatena 'Title' e 'Detail' nel secondo dataset
texts2 = dataset2['Title'].fillna('') + ' ' + dataset2['Detail'].fillna('')
texts2 = texts2[texts2.str.strip() != '']

# Unisci i due datasets di testo (puoi usare un'unica variabile per combinarli)
texts= pd.concat([texts1, texts2], ignore_index=True)

#texts è solo il primo 
texts=texts1


# Funzione per ottenere il sentiment per ogni testo
def get_sentiment(text, true_sentiment):
    # Tokenizza il testo
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Invia il testo attraverso il modello
    with torch.no_grad():
        outputs = model(**inputs)

    # Ottieni le probabilità di sentiment (5 classi da 0 a 4)
    sentiment_scores = outputs.logits
    sentiment = torch.argmax(sentiment_scores, dim=1).item()  # Ottieni la classe con la probabilità più alta

    sentiment = sentiment + 1  # Mappa da 0-4 a 1-5

    return {'text': text, 'sentiment': sentiment, 'trueSentiment': true_sentiment}

# Applica la funzione di sentiment analysis a tutti i testi
sentiments = [get_sentiment(text, true_sentiment) for text, true_sentiment in zip(texts, dataset1['feedbackRating'])]

print(sentiments)

# Converte i risultati in un DataFrame
#sentiments_df = pd.DataFrame(sentiments)

# Calcola l'accuratezza: confronta sentiment e trueSentiment
#correct_predictions = (sentiments_df['sentiment'] == sentiments_df['trueSentiment']).sum()
#accuracy = correct_predictions / len(sentiments_df) * 100

# Calcola l'accuracy off-by-one: quando la differenza tra sentiment e trueSentiment è 1
#accuracy_off_by_one = (abs(sentiments_df['trueSentiment'] - sentiments_df['sentiment']) <= 1).sum()
#accuracy_off_by_one_percent = accuracy_off_by_one / len(sentiments_df) * 100


# Stampa l'accuratezza e l'accuracy-1
#print(f"Accuracy: {accuracy:.2f}%")
#print(f"Accuracy off-by-one: {accuracy_off_by_one_percent:.2f}%")

#accuracy 30%, off by one 65.4%

## Sentiment Classifier 

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score

tokenizer = AutoTokenizer.from_pretrained("lxyuan/distilbert-base-multilingual-cased-sentiments-student")
model = AutoModelForSequenceClassification.from_pretrained("lxyuan/distilbert-base-multilingual-cased-sentiments-student")

#questo print ci fa sapere queli sono le label 
#print(model.config.id2label)

# Carica il secondo dataset
dataset2 = pd.read_csv("Tiny Eco Digimind.csv", sep=";", on_bad_lines="skip")
dataset2.columns = dataset2.columns.str.strip()

texts2= texts2.str.replace(r'\s+', ' ', regex=True)  # Rimuove spazi extra (compresi \n e \t)
texts2 = texts2.str.strip().str.replace('\n', ' ', regex=True)

trueSentiment2 = dataset2['Sentiment'].str.strip()

def get_sentiment(text, true_sentiment):
    # Tokenizza il testo
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Invia il testo attraverso il modello
    with torch.no_grad():
        outputs = model(**inputs)

    # Ottieni le probabilità di sentiment (5 classi da 0 a 4)
    sentiment_scores = outputs.logits
    sentiment_numeric = torch.argmax(sentiment_scores, dim=1).item()  # Ottieni la classe con la probabilità più alta

    # Mappa numeri a etichette
    sentiment_map = {
        0: "Positive",
        1: "No sentiment",
        2: "Negative"
    }
    sentiment = sentiment_map.get(sentiment_numeric, "Unknown")  # Default a 'Unknown' per valori imprevisti

    return {'sentiment': sentiment, 'trueSentiment': true_sentiment}


# Applica la funzione di sentiment analysis a tutti i testi
sentiments2 = [get_sentiment(text2, true_sentiment) for text2, true_sentiment in zip(texts2, trueSentiment2)]

# Estrai le predizioni e i sentimenti veri
predicted_sentiments = [item['sentiment'] for item in sentiments2]
true_sentiments = list(trueSentiment2)

# Calcola l'accuracy
accuracy = accuracy_score(true_sentiments, predicted_sentiments)

# Stampa il risultato
print(f"Accuracy: {accuracy:.2%}")


This model has an accuracy of 35.01%, now we will evaluate another sentiment classifier and see which one works better.

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score

# Verifica se la GPU è disponibile
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

tokenizer = AutoTokenizer.from_pretrained("citizenlab/twitter-xlm-roberta-base-sentiment-finetunned")
model = AutoModelForSequenceClassification.from_pretrained("citizenlab/twitter-xlm-roberta-base-sentiment-finetunned")

#print(model.config.id2label)


# Carica il secondo dataset
dataset2 = pd.read_csv("Tiny Eco Digimind.csv", sep=";", on_bad_lines="skip")
dataset2.columns = dataset2.columns.str.strip()

texts2= texts2.str.replace(r'\s+', ' ', regex=True)  # Rimuove spazi extra (compresi \n e \t)
texts2 = texts2.str.strip().str.replace('\n', ' ', regex=True)

trueSentiment2 = dataset2['Sentiment'].str.strip()

def get_sentiment(text, true_sentiment):
    # Tokenizza il testo
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Invia il testo attraverso il modello
    with torch.no_grad():
        outputs = model(**inputs)

    # Ottieni le probabilità di sentiment (5 classi da 0 a 4)
    sentiment_scores = outputs.logits
    sentiment_numeric = torch.argmax(sentiment_scores, dim=1).item()  # Ottieni la classe con la probabilità più alta

    # Mappa numeri a etichette
    sentiment_map = {
        0: "Negative",
        1: "No sentiment",
        2: "Positive"
    }
    sentiment = sentiment_map.get(sentiment_numeric, "Unknown")  # Default a 'Unknown' per valori imprevisti

    return {'sentiment': sentiment, 'trueSentiment': true_sentiment}

# Applica la funzione di sentiment analysis a tutti i testi
sentiments3 = [get_sentiment(text2, true_sentiment) for text2, true_sentiment in zip(texts2, trueSentiment2)]

print(sentiments3)

# Estrai le predizioni e i sentimenti veri
predicted_sentiments = [item['sentiment'] for item in sentiments3]
true_sentiments = list(trueSentiment2)

# Calcola l'accuracy
accuracy = accuracy_score(true_sentiments, predicted_sentiments)

# Stampa il risultato
print(f"Accuracy: {accuracy:.2%}")

This model has an accuracy of 59.97%

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd

# Verifica se la GPU è disponibile
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Carica il tokenizer e il modello di traduzione (Italiano -> Inglese)
translation_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-it-en", use_fast=False)
translation_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-it-en").to(device)

# Carica il dataset
dataset2 = pd.read_csv("Tiny Eco Digimind.csv", sep=";", on_bad_lines="skip")
dataset2.columns = dataset2.columns.str.strip()

# Prepara i testi
texts2 = dataset2['Title'].fillna('') + ' ' + dataset2['Detail'].fillna('')
texts2 = texts2.str.replace(r'\s+', ' ', regex=True).str.strip().str.replace('\n', ' ', regex=True)

# Funzione per la traduzione
def translate_text_it_to_en(text):
    # Tokenizza il testo per la traduzione
    inputs = translation_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    # Invia il testo al modello di traduzione
    with torch.no_grad():
        translated_ids = translation_model.generate(**inputs)

    # Decodifica la traduzione (da ID a testo)
    translated_text = translation_tokenizer.decode(translated_ids[0], skip_special_tokens=True)
    return translated_text

# Tradurre tutte le frasi
translated_texts = [translate_text_it_to_en(text) for text in texts2]

# Crea un DataFrame con le frasi originali e tradotte
translation_df = pd.DataFrame({
    'Original': texts2,
    'Translated': translated_texts
})

# Stampa la matrice delle frasi originali e tradotte
print(translation_df)


Using device: cuda


C:\Users\vcata\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Tokenizer caricato correttamente!


C:\Users\vcata\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vcata\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-it-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [2]:
# Verifica se la GPU è disponibile
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Carica il tokenizer e il modello per la sentiment analysis
sentiment_tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
sentiment_model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest").to(device)
# Funzione per l'analisi del sentiment
def analyze_sentiment(translated_text, true_sentiment):
    # Tokenizza il testo tradotto per l'analisi del sentiment
    inputs = sentiment_tokenizer(translated_text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)

    # Invia il testo al modello di sentiment analysis
    with torch.no_grad():
        outputs = sentiment_model(**inputs)

    # Ottieni le probabilità di sentiment (classi: 0 = Negative, 1 = No sentiment, 2 = Positive)
    sentiment_scores = outputs.logits
    sentiment_numeric = torch.argmax(sentiment_scores, dim=1).item()  # Ottieni la classe con la probabilità più alta

    # Mappa numeri a etichette
    sentiment_map = {
        0: "Negative",
        1: "No sentiment",
        2: "Positive"
    }
    sentiment = sentiment_map.get(sentiment_numeric, "Unknown")  # Default a 'Unknown' per valori imprevisti

    return {'sentiment': sentiment, 'trueSentiment': true_sentiment}

sentiments4 = [analyze_sentiment(translated_text, true_sentiment) for translated_text, true_sentiment in zip(translated_texts, trueSentiment2)]

print(sentiments4)

# Estrai le predizioni e i sentimenti veri
predicted_sentiments = [item['sentiment'] for item in sentiments4]
true_sentiments = list(trueSentiment2)

# Calcola l'accuracy
accuracy = accuracy_score(true_sentiments, predicted_sentiments)

# Stampa il risultato
print(f"Accuracy: {accuracy:.2%}")


Using device: cuda


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyboardInterrupt: 

In [ ]:
import torch

# Verifica se CUDA è disponibile
print(torch.cuda.is_available())

# Mostra il numero di GPU disponibili (se CUDA è disponibile)
print(torch.cuda.device_count())

# Nome della GPU in uso (se CUDA è disponibile)
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
else:
    print("CUDA non è disponibile")
